In [1]:
import os

import pandas as pd
import geopandas as gpd
from shapely.geometry import MultiPoint
from shapely import wkt
# from sqlalchemy import text

import mripython as mri

### Params

In [2]:
FP_NODES='s3://gi-rawdata/osm/uk/nodes/nodes_20200520.parquet'
FP_WAYS='s3://gi-rawdata/osm/uk/ways/ways_20200519.csv'
FP_POSTCODE_BUFFER='s3://gidev/projects/open-street-map/uk/postcodes/geodata/uk_postcodes_wkt_buffer_250m_20200602.csv'

FP_LOCAL='/home/selina/local_meg/nick/nudism_tags.csv'

### Load and subset OSM data

#### Ways

In [3]:
%%time
ways=pd.read_csv(FP_WAYS)

CPU times: user 1min 13s, sys: 14.9 s, total: 1min 28s
Wall time: 2min 21s


In [4]:
ways.head()

,id,tags,lat,lon
0,85478037,"{'name': 'Ridgeway', 'source': 'OS_OpenData_St...","[50.2906706, 50.2903002, 50.2910075, 50.291058...","[-4.9505771, -4.9502946, -4.9510263, -4.951095..."
1,123355265,{'highway': 'service'},"[52.6923470, 52.6926188, 52.6925987, 52.692393...","[-1.7144764, -1.7116259, -1.7113977, -1.714894..."
2,34870294,"{'name': 'Dylan Road', 'highway': 'residential'}","[51.4918822, 51.4916098]","[0.1493202, 0.1492267]"
3,123372305,{'landuse': 'allotments'},"[50.8336562, 50.8344716, 50.8343504, 50.834497...","[-0.1050033, -0.1025730, -0.1022406, -0.102829..."
4,123372456,"{'name': 'Outwoods', 'landuse': 'forest'}","[52.5704082, 52.5716495, 52.5704890, 52.566637...","[-1.5548786, -1.5571366, -1.5543511, -1.560078..."


In [5]:
ways['nudism_tag']=ways['tags'].apply(lambda x:eval(x).get('nudism'))

In [6]:
ways.head()

,id,tags,lat,lon,nudism_tag
0,85478037,"{'name': 'Ridgeway', 'source': 'OS_OpenData_St...","[50.2906706, 50.2903002, 50.2910075, 50.291058...","[-4.9505771, -4.9502946, -4.9510263, -4.951095...",None
1,123355265,{'highway': 'service'},"[52.6923470, 52.6926188, 52.6925987, 52.692393...","[-1.7144764, -1.7116259, -1.7113977, -1.714894...",None
2,34870294,"{'name': 'Dylan Road', 'highway': 'residential'}","[51.4918822, 51.4916098]","[0.1493202, 0.1492267]",None
3,123372305,{'landuse': 'allotments'},"[50.8336562, 50.8344716, 50.8343504, 50.834497...","[-0.1050033, -0.1025730, -0.1022406, -0.102829...",None
4,123372456,"{'name': 'Outwoods', 'landuse': 'forest'}","[52.5704082, 52.5716495, 52.5704890, 52.566637...","[-1.5548786, -1.5571366, -1.5543511, -1.560078...",None


In [7]:
ways.nudism_tag.notnull().mean()

9.251059273495813e-07

In [8]:
ways.nudism_tag.notnull().sum()

17

In [9]:
ways=ways[ways.nudism_tag.notnull()]

In [10]:
ways

,id,tags,lat,lon,nudism_tag
430130,338490982,"{'nudism': 'customary', 'natural': 'beach', 's...","[55.4384074, 55.4380560, 55.4389051, 55.438807...","[-5.2469849, -5.2377426, -5.2437776, -5.245606...",customary
4166666,230388260,"{'nudism': 'yes', 'natural': 'beach', 'surface...","[50.6629957, 50.6606995, 50.6646572, 50.661110...","[-1.9487428, -1.9502858, -1.9475646, -1.950018...",yes
5296449,457666787,"{'nudism': 'designated', 'natural': 'beach', '...","[55.4394703, 55.4391214, 55.4391547, 55.439392...","[-5.2514198, -5.2504825, -5.2482859, -5.249379...",designated
5504328,96624951,"{'nudism': 'no', 'source:name': 'local_knowled...","[50.8170231, 50.8161311, 50.8187325, 50.819078...","[-0.1237390, -0.1201978, -0.1320089, -0.133979...",no
5631240,457668845,"{'nudism': 'designated', 'natural': 'beach', '...","[55.6179708, 55.6133075, 55.6144670, 55.608728...","[-4.7227414, -4.7117769, -4.7149181, -4.702394...",designated
6339863,168472202,"{'nudism': 'yes', 'natural': 'beach', 'surface...","[50.7868400, 50.7870341, 50.7870883, 50.786557...","[-1.0309351, -1.0308962, -1.0304885, -1.032078...",yes
6348725,367488137,"{'nudism': 'designated', 'natural': 'beach', '...","[55.6180147, 55.6203397, 55.6232237, 55.618868...","[-4.7234138, -4.7302037, -4.7375026, -4.725387...",designated
7634632,172581555,"{'nudism': 'yes', 'phone': '+44 1362858871', '...","[52.6759496, 52.6759691, 52.6759691, 52.677537...","[1.0071964, 1.0081888, 1.0081888, 1.0073828, 1...",yes
8398454,168472207,"{'nudism': 'yes', 'natural': 'beach', 'name': ...","[51.3758713, 51.3767414, 51.3785798, 51.379209...","[0.9489995, 0.9476394, 0.9455351, 0.9450513, 0...",yes
8694998,168472197,"{'nudism': 'no', 'natural': 'beach', 'source':...","[50.8134502, 50.8128533, 50.8133697, 50.813928...","[-0.1094882, -0.1075543, -0.1085115, -0.109789...",no


In [32]:
ways['tags']

430130      {'nudism': 'customary', 'natural': 'beach', 's...
4166666     {'nudism': 'yes', 'natural': 'beach', 'surface...
5296449     {'nudism': 'designated', 'natural': 'beach', '...
5504328     {'nudism': 'no', 'source:name': 'local_knowled...
5631240     {'nudism': 'designated', 'natural': 'beach', '...
6339863     {'nudism': 'yes', 'natural': 'beach', 'surface...
6348725     {'nudism': 'designated', 'natural': 'beach', '...
7634632     {'nudism': 'yes', 'phone': '+44 1362858871', '...
8398454     {'nudism': 'yes', 'natural': 'beach', 'name': ...
8694998     {'nudism': 'no', 'natural': 'beach', 'source':...
10284036    {'nudism': 'yes', 'natural': 'beach', 'name': ...
11620085    {'note': 'Was formerly mapped as a wood with a...
11679808    {'website': 'https://www.naturistfoundation.or...
12599012    {'nudism': 'yes', 'natural': 'beach', 'name': ...
14736157    {'nudism': 'permissive', 'natural': 'beach', '...
15756509    {'nudism': 'designated', 'website': 'http://ww...
16253134

In [34]:
ways=ways.loc[ways.tags.notnull(),['id','tags','lat','lon','nudism_tag']]

In [35]:
ways.shape

(17, 5)

In [36]:
ways = pd.concat([ways, ways.tags.apply(lambda x:eval(x)).apply(pd.Series)], axis=1, sort=False)

In [37]:
ways.notnull().mean()

id                 1.000000
tags               1.000000
lat                1.000000
lon                1.000000
nudism_tag         1.000000
nudism             1.000000
natural            0.705882
surface            0.352941
name               0.882353
tourism            0.294118
source:name        0.058824
beach              0.058824
source             0.235294
url                0.235294
description        0.176471
phone              0.058824
note               0.058824
website            0.235294
access             0.117647
club               0.058824
landuse            0.058824
leisure            0.058824
wikidata           0.058824
operator           0.117647
long_name          0.058824
addr:country       0.058824
motorhome          0.058824
addr:postcode      0.058824
dryer              0.058824
tents              0.117647
addr:city          0.058824
power_supply       0.058824
sauna              0.058824
caravans           0.058824
addr:street        0.058824
washing_machine    0

In [38]:
ways.shape

(17, 39)

In [39]:
ways[ways.nudism_tag=='obligatory']

,id,tags,lat,lon,nudism_tag,nudism,natural,surface,name,tourism,...,tents,addr:city,power_supply,sauna,caravans,addr:street,washing_machine,source:nudist,nudist,barrier
11620085,487400097.0,{'note': 'Was formerly mapped as a wood with a...,"[51.7174211, 51.7170612, 51.7165404, 51.717613...","[-0.3588151, -0.3551418, -0.3573348, -0.357190...",obligatory,obligatory,NaN,NaN,Spielplatz,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
ways.columns

Index(['id', 'tags', 'lat', 'lon', 'nudism_tag', 'nudism', 'natural',
       'surface', 'name', 'tourism', 'source:name', 'beach', 'source', 'url',
       'description', 'phone', 'note', 'website', 'access', 'club', 'landuse',
       'leisure', 'wikidata', 'operator', 'long_name', 'addr:country',
       'motorhome', 'addr:postcode', 'dryer', 'tents', 'addr:city',
       'power_supply', 'sauna', 'caravans', 'addr:street', 'washing_machine',
       'source:nudist', 'nudist', 'barrier'],
      dtype='object')

In [42]:
ways[['nudism','natural','name','tourism','beach','source','addr:postcode']]

,nudism,natural,name,tourism,beach,source,addr:postcode
430130,customary,beach,NaN,NaN,NaN,NaN,NaN
4166666,yes,beach,Studland Naturist Beach,attraction,NaN,NaN,NaN
5296449,designated,beach,Cleats Shore,NaN,NaN,NaN,NaN
5504328,no,beach,Brighton Beach,NaN,pebble,NPE,NaN
5631240,designated,beach,Ardeer Beach,NaN,NaN,NaN,NaN
6339863,yes,beach,Eastney Beach,NaN,NaN,os_opendata_streetview,NaN
6348725,designated,beach,Ardeer Beach,NaN,NaN,NaN,NaN
7634632,yes,NaN,Abbott's Farm (Naturist),camp_site,NaN,NaN,NaN
8398454,yes,beach,Leysdown East,NaN,NaN,NaN,NaN
8694998,no,beach,NaN,NaN,NaN,NPE,NaN


#### Nodes

In [43]:
nodes = pd.read_parquet(FP_NODES)

In [44]:
nodes.head()

,id,tags,lat,lon
0,148526412,{'created_by': 'JOSM'},55.006420,-1.641230
1,148526417,{'created_by': 'JOSM'},55.006074,-1.641132
2,148526763,"{'highway': 'crossing', 'crossing': 'toucan'}",50.535192,-3.604767
3,148540085,"{'name': 'Frodsham Water Sports Centre', 'tour...",53.300373,-2.706535
4,148541836,{'created_by': 'JOSM'},51.514881,-0.740449


In [45]:
nodes['nudism_tag']=nodes['tags'].apply(lambda x:eval(x).get('nudism'))

In [47]:
nodes.nudism_tag.notnull().mean()

2.09663605228591e-06

In [48]:
nodes=nodes[nodes.nudism_tag.notnull()]

In [49]:
nodes.shape

(10, 5)

In [50]:
nodes

,id,tags,lat,lon,nudism_tag
1708525,1723311196,"{'wheelchair': 'limited', 'nudism': 'yes', 'na...",50.408038,1.559925,yes
1708526,1723311330,"{'nudism': 'yes', 'natural': 'beach', 'name': ...",50.031651,1.305681,yes
1708527,1723311454,"{'nudism': 'yes', 'natural': 'beach', 'name': ...",50.877254,1.638539,yes
1708528,1723311510,"{'nudism': 'yes', 'natural': 'beach', 'name': ...",50.642635,1.576490,yes
1708529,1723311770,"{'nudism': 'yes', 'natural': 'beach', 'name': ...",50.271592,1.539084,yes
1708530,1723311813,"{'nudism': 'yes', 'natural': 'beach', 'name': ...",50.503071,1.577925,yes
3178632,5658513387,{'note': 'Not an official nudist beachbut loca...,51.100464,1.249234,customary
3349513,1797360071,"{'nudism': 'yes', 'natural': 'beach', 'name': ...",50.620526,-3.348856,yes
3349514,1797360076,"{'nudism': 'yes', 'natural': 'beach', 'name': ...",52.971897,0.806266,yes
3349515,1797360078,"{'nudism': 'permissive', 'natural': 'beach', '...",50.828107,-0.222249,permissive


In [52]:
nodes = pd.concat([nodes, nodes.tags.apply(lambda x:eval(x)).apply(pd.Series)], axis=1, sort=False)

In [53]:
nodes.shape

(10, 11)

In [54]:
nodes

,id,tags,lat,lon,nudism_tag,wheelchair,nudism,natural,name,note,url
1708525,1723311196,"{'wheelchair': 'limited', 'nudism': 'yes', 'na...",50.408038,1.559925,yes,limited,yes,beach,Berck Plage,NaN,NaN
1708526,1723311330,"{'nudism': 'yes', 'natural': 'beach', 'name': ...",50.031651,1.305681,yes,NaN,yes,beach,Criel sur mer,NaN,NaN
1708527,1723311454,"{'nudism': 'yes', 'natural': 'beach', 'name': ...",50.877254,1.638539,yes,NaN,yes,beach,Dunes du Chatelet,NaN,NaN
1708528,1723311510,"{'nudism': 'yes', 'natural': 'beach', 'name': ...",50.642635,1.576490,yes,NaN,yes,beach,Hardelot Plage,NaN,NaN
1708529,1723311770,"{'nudism': 'yes', 'natural': 'beach', 'name': ...",50.271592,1.539084,yes,NaN,yes,beach,Plage du Domaine de Marquenterre,NaN,NaN
1708530,1723311813,"{'nudism': 'yes', 'natural': 'beach', 'name': ...",50.503071,1.577925,yes,NaN,yes,beach,Stella Plage,NaN,NaN
3178632,5658513387,{'note': 'Not an official nudist beachbut loca...,51.100464,1.249234,customary,NaN,customary,beach,Abbot's Cliff Beach,Not an official nudist beachbut local knowledg...,NaN
3349513,1797360071,"{'nudism': 'yes', 'natural': 'beach', 'name': ...",50.620526,-3.348856,yes,NaN,yes,beach,Budleigh Salterton,NaN,http://www.naturistholiday.info/UK/1240/Budlei...
3349514,1797360076,"{'nudism': 'yes', 'natural': 'beach', 'name': ...",52.971897,0.806266,yes,NaN,yes,beach,Holkham Naturist Beach,NaN,http://www.naturistholiday.info/UK/1245/Holkha...
3349515,1797360078,"{'nudism': 'permissive', 'natural': 'beach', '...",50.828107,-0.222249,permissive,NaN,permissive,beach,Portslade,NaN,http://www.naturistholiday.info/UK/1974/Portsl...


### Spatial join to postcodes

#### Create ways geodf

In [80]:
ways['geometry'] = ways.apply(lambda x: MultiPoint(list(zip(eval(x['lon']),
                                                            eval(x['lat'])
                                                           )
                                                       )
                                                  ),axis=1)

In [81]:
ways=gpd.GeoDataFrame(ways, geometry='geometry', crs='EPSG:4326')

In [82]:
ways.head()

,id,tags,lat,lon,nudism_tag,nudism,natural,surface,name,tourism,...,addr:city,power_supply,sauna,caravans,addr:street,washing_machine,source:nudist,nudist,barrier,geometry
430130,338490982.0,"{'nudism': 'customary', 'natural': 'beach', 's...","[55.4384074, 55.4380560, 55.4389051, 55.438807...","[-5.2469849, -5.2377426, -5.2437776, -5.245606...",customary,customary,beach,pebblestone,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(POINT (-5.2469849 55.4384074), POINT (-5.2377..."
4166666,230388260.0,"{'nudism': 'yes', 'natural': 'beach', 'surface...","[50.6629957, 50.6606995, 50.6646572, 50.661110...","[-1.9487428, -1.9502858, -1.9475646, -1.950018...",yes,yes,beach,sand,Studland Naturist Beach,attraction,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(POINT (-1.9487428 50.6629957), POINT (-1.9502..."
5296449,457666787.0,"{'nudism': 'designated', 'natural': 'beach', '...","[55.4394703, 55.4391214, 55.4391547, 55.439392...","[-5.2514198, -5.2504825, -5.2482859, -5.249379...",designated,designated,beach,sand,Cleats Shore,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(POINT (-5.2514198 55.4394703), POINT (-5.2504..."
5504328,96624951.0,"{'nudism': 'no', 'source:name': 'local_knowled...","[50.8170231, 50.8161311, 50.8187325, 50.819078...","[-0.1237390, -0.1201978, -0.1320089, -0.133979...",no,no,beach,NaN,Brighton Beach,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(POINT (-0.123739 50.8170231), POINT (-0.12019..."
5631240,457668845.0,"{'nudism': 'designated', 'natural': 'beach', '...","[55.6179708, 55.6133075, 55.6144670, 55.608728...","[-4.7227414, -4.7117769, -4.7149181, -4.702394...",designated,designated,beach,sand,Ardeer Beach,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(POINT (-4.7227414 55.6179708), POINT (-4.7117..."


#### Create nodes geodf

In [83]:
nodes = gpd.GeoDataFrame(nodes, geometry=gpd.points_from_xy(nodes['lon'],
                                                            nodes['lat']
                                                           ), crs='EPSG:4326'
                        )

In [84]:
nodes.shape

(10, 12)

In [85]:
nodes.head()

,id,tags,lat,lon,nudism_tag,wheelchair,nudism,natural,name,note,url,geometry
1708525,1723311196,"{'wheelchair': 'limited', 'nudism': 'yes', 'na...",50.408038,1.559925,yes,limited,yes,beach,Berck Plage,NaN,NaN,POINT (1.5599249 50.4080376)
1708526,1723311330,"{'nudism': 'yes', 'natural': 'beach', 'name': ...",50.031651,1.305681,yes,NaN,yes,beach,Criel sur mer,NaN,NaN,POINT (1.305681 50.031651)
1708527,1723311454,"{'nudism': 'yes', 'natural': 'beach', 'name': ...",50.877254,1.638539,yes,NaN,yes,beach,Dunes du Chatelet,NaN,NaN,POINT (1.638539 50.877254)
1708528,1723311510,"{'nudism': 'yes', 'natural': 'beach', 'name': ...",50.642635,1.576490,yes,NaN,yes,beach,Hardelot Plage,NaN,NaN,POINT (1.57649 50.642635)
1708529,1723311770,"{'nudism': 'yes', 'natural': 'beach', 'name': ...",50.271592,1.539084,yes,NaN,yes,beach,Plage du Domaine de Marquenterre,NaN,NaN,POINT (1.539084 50.27159210000001)


#### Create postcode geodf

In [86]:
uk_postcodes = pd.read_csv(FP_POSTCODE_BUFFER)

In [87]:
uk_postcodes.head()

,postcode,buffer
0,AB1 0AA,POLYGON ((-2.240605211789701 57.10147399999729...
1,AB1 0AB,POLYGON ((-2.244062211789701 57.10255399999728...
2,AB1 0AD,POLYGON ((-2.246096211789701 57.10055599999728...
3,AB1 0AE,POLYGON ((-2.253462211789701 57.08444399999727...
4,AB1 0AF,POLYGON ((-2.255856211789701 57.09665599999727...


In [89]:
uk_postcodes.dtypes

postcode    object
buffer      object
dtype: object

In [90]:
uk_postcodes['geometry']=uk_postcodes['buffer'].apply(wkt.loads) 

That took a while - 5 mins 37.19 s !

In [92]:
%%time
uk_postcodes = gpd.GeoDataFrame(uk_postcodes, geometry='geometry', crs='EPSG:4326')

CPU times: user 712 ms, sys: 48 ms, total: 760 ms
Wall time: 756 ms


Relatively quick, I guess loading the wkt took a while

#### Spatial joins

In [97]:
%%time
ways_gdf = gpd.sjoin(uk_postcodes, ways, how='right', op='intersects').drop(['index_left'],axis=1)

CPU times: user 12.3 s, sys: 260 ms, total: 12.6 s
Wall time: 12.6 s


In [98]:
ways_gdf.head()

,postcode,buffer,id,tags,lat,lon,nudism_tag,nudism,natural,surface,...,addr:city,power_supply,sauna,caravans,addr:street,washing_machine,source:nudist,nudist,barrier,geometry
5504328,BN1 1GF,POLYGON ((-0.1341982117897012 50.8202829999902...,96624951.0,"{'nudism': 'no', 'source:name': 'local_knowled...","[50.8170231, 50.8161311, 50.8187325, 50.819078...","[-0.1237390, -0.1201978, -0.1320089, -0.133979...",no,no,beach,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(POINT (-0.123739 50.8170231), POINT (-0.12019..."
5504328,BN2 1TN,POLYGON ((-0.1320932117897012 50.8203929999902...,96624951.0,"{'nudism': 'no', 'source:name': 'local_knowled...","[50.8170231, 50.8161311, 50.8187325, 50.819078...","[-0.1237390, -0.1201978, -0.1320089, -0.133979...",no,no,beach,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(POINT (-0.123739 50.8170231), POINT (-0.12019..."
5504328,BN2 1TE,POLYGON ((-0.1338522117897012 50.8204339999902...,96624951.0,"{'nudism': 'no', 'source:name': 'local_knowled...","[50.8170231, 50.8161311, 50.8187325, 50.819078...","[-0.1237390, -0.1201978, -0.1320089, -0.133979...",no,no,beach,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(POINT (-0.123739 50.8170231), POINT (-0.12019..."
5504328,BN2 1TY,POLYGON ((-0.1337232117897012 50.8204549999902...,96624951.0,"{'nudism': 'no', 'source:name': 'local_knowled...","[50.8170231, 50.8161311, 50.8187325, 50.819078...","[-0.1237390, -0.1201978, -0.1320089, -0.133979...",no,no,beach,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(POINT (-0.123739 50.8170231), POINT (-0.12019..."
5504328,BN1 1EL,POLYGON ((-0.1340912117897012 50.8205009999902...,96624951.0,"{'nudism': 'no', 'source:name': 'local_knowled...","[50.8170231, 50.8161311, 50.8187325, 50.819078...","[-0.1237390, -0.1201978, -0.1320089, -0.133979...",no,no,beach,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(POINT (-0.123739 50.8170231), POINT (-0.12019..."


In [100]:
ways_gdf.shape

(125, 42)

In [101]:
ways_gdf.id.nunique()

17

In [103]:
ways_gdf.columns

Index(['postcode', 'buffer', 'id', 'tags', 'lat', 'lon', 'nudism_tag',
       'nudism', 'natural', 'surface', 'name', 'tourism', 'source:name',
       'beach', 'source', 'url', 'description', 'phone', 'note', 'website',
       'access', 'club', 'landuse', 'leisure', 'wikidata', 'operator',
       'long_name', 'addr:country', 'motorhome', 'addr:postcode', 'dryer',
       'tents', 'addr:city', 'power_supply', 'sauna', 'caravans',
       'addr:street', 'washing_machine', 'source:nudist', 'nudist', 'barrier',
       'geometry'],
      dtype='object')

In [105]:
%%time
nodes_gdf = gpd.sjoin(uk_postcodes, nodes, how='right', op='intersects').drop(['index_left'],axis=1)

CPU times: user 12.3 s, sys: 296 ms, total: 12.6 s
Wall time: 12.6 s


In [106]:
nodes_gdf.shape

(11, 14)

In [107]:
nodes_gdf.id.nunique()

10

### Combine data

In [108]:
ways_nodes=pd.concat([ways_gdf,nodes_gdf], axis=0, sort=False, ignore_index=True)

In [109]:
ways_nodes.head()

,postcode,buffer,id,tags,lat,lon,nudism_tag,nudism,natural,surface,...,power_supply,sauna,caravans,addr:street,washing_machine,source:nudist,nudist,barrier,geometry,wheelchair
0,BN1 1GF,POLYGON ((-0.1341982117897012 50.8202829999902...,96624951.0,"{'nudism': 'no', 'source:name': 'local_knowled...","[50.8170231, 50.8161311, 50.8187325, 50.819078...","[-0.1237390, -0.1201978, -0.1320089, -0.133979...",no,no,beach,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(POINT (-0.123739 50.8170231), POINT (-0.12019...",NaN
1,BN2 1TN,POLYGON ((-0.1320932117897012 50.8203929999902...,96624951.0,"{'nudism': 'no', 'source:name': 'local_knowled...","[50.8170231, 50.8161311, 50.8187325, 50.819078...","[-0.1237390, -0.1201978, -0.1320089, -0.133979...",no,no,beach,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(POINT (-0.123739 50.8170231), POINT (-0.12019...",NaN
2,BN2 1TE,POLYGON ((-0.1338522117897012 50.8204339999902...,96624951.0,"{'nudism': 'no', 'source:name': 'local_knowled...","[50.8170231, 50.8161311, 50.8187325, 50.819078...","[-0.1237390, -0.1201978, -0.1320089, -0.133979...",no,no,beach,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(POINT (-0.123739 50.8170231), POINT (-0.12019...",NaN
3,BN2 1TY,POLYGON ((-0.1337232117897012 50.8204549999902...,96624951.0,"{'nudism': 'no', 'source:name': 'local_knowled...","[50.8170231, 50.8161311, 50.8187325, 50.819078...","[-0.1237390, -0.1201978, -0.1320089, -0.133979...",no,no,beach,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(POINT (-0.123739 50.8170231), POINT (-0.12019...",NaN
4,BN1 1EL,POLYGON ((-0.1340912117897012 50.8205009999902...,96624951.0,"{'nudism': 'no', 'source:name': 'local_knowled...","[50.8170231, 50.8161311, 50.8187325, 50.819078...","[-0.1237390, -0.1201978, -0.1320089, -0.133979...",no,no,beach,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(POINT (-0.123739 50.8170231), POINT (-0.12019...",NaN


In [110]:
ways_nodes.id.nunique()

27

In [111]:
ways_nodes.shape

(136, 43)

#### Group by id

In [115]:
ways_nodes.columns

Index(['postcode', 'buffer', 'id', 'tags', 'lat', 'lon', 'nudism_tag',
       'nudism', 'natural', 'surface', 'name', 'tourism', 'source:name',
       'beach', 'source', 'url', 'description', 'phone', 'note', 'website',
       'access', 'club', 'landuse', 'leisure', 'wikidata', 'operator',
       'long_name', 'addr:country', 'motorhome', 'addr:postcode', 'dryer',
       'tents', 'addr:city', 'power_supply', 'sauna', 'caravans',
       'addr:street', 'washing_machine', 'source:nudist', 'nudist', 'barrier',
       'geometry', 'wheelchair'],
      dtype='object')

In [122]:
ways_nodes = ways_nodes.groupby('id', as_index=False).agg({'postcode':lambda x:list(x),
                                     'tags':'first',
                                     'lat':'first',
                                     'lon':'first',
                                     'nudism':'first', 
                                     'natural':'first', 
                                     'surface':'first', 
                                     'name':'first', 
                                     'tourism':'first', 
                                     'source:name':'first',
                                     'beach':'first', 
                                     'source':'first', 
                                     'url':'first', 
                                     'description':'first', 
                                     'phone':'first', 
                                     'note':'first', 
                                     'website':'first', 
                                     'access':'first', 
                                     'club':'first',
                                     'landuse':'first',
                                     'leisure':'first', 
                                     'wikidata':'first', 
                                     'operator':'first',
                                     'long_name':'first',
                                     'addr:country':'first', 
                                     'motorhome':'first', 
                                     'addr:postcode':'first',
                                     'dryer':'first',
                                     'tents':'first', 
                                     'addr:city':'first', 
                                     'power_supply':'first',
                                     'sauna':'first', 
                                     'caravans':'first',
                                     'addr:street':'first', 
                                     'washing_machine':'first', 
                                     'source:nudist':'first', 
                                     'nudist':'first', 
                                     'barrier':'first',
                                     'geometry':'first', 
                                     'wheelchair':'first'
                                    })

In [123]:
ways_nodes.head()

,id,postcode,tags,lat,lon,nudism,natural,surface,name,tourism,...,power_supply,sauna,caravans,addr:street,washing_machine,source:nudist,nudist,barrier,geometry,wheelchair
0,96624951.0,"[BN1 1GF, BN2 1TN, BN2 1TE, BN2 1TY, BN1 1EL, ...","{'nudism': 'no', 'source:name': 'local_knowled...","[50.8170231, 50.8161311, 50.8187325, 50.819078...","[-0.1237390, -0.1201978, -0.1320089, -0.133979...",no,beach,NaN,Brighton Beach,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(POINT (-0.123739 50.8170231), POINT (-0.12019...",NaN
1,166854264.0,[nan],"{'nudism': 'permissive', 'natural': 'beach', '...","[50.4886730, 50.4879567, 50.4886730, 50.487741...","[-3.5144732, -3.5142425, -3.5144732, -3.514711...",permissive,beach,NaN,Petit Tor Beach,NaN,...,NaN,NaN,NaN,NaN,NaN,OMG,permissive,NaN,"(POINT (-3.5144732 50.488673), POINT (-3.51424...",NaN
2,168472197.0,"[BN2 5UE, BN2 5TT, BN2 5TU, BN2 5TY, BN2 5UD, ...","{'nudism': 'no', 'natural': 'beach', 'source':...","[50.8134502, 50.8128533, 50.8133697, 50.813928...","[-0.1094882, -0.1075543, -0.1085115, -0.109789...",no,beach,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(POINT (-0.1094882 50.8134502), POINT (-0.1075...",NaN
3,168472202.0,[PO4 9LJ],"{'nudism': 'yes', 'natural': 'beach', 'surface...","[50.7868400, 50.7870341, 50.7870883, 50.786557...","[-1.0309351, -1.0308962, -1.0304885, -1.032078...",yes,beach,pebbles,Eastney Beach,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(POINT (-1.0309351 50.78684), POINT (-1.030896...",NaN
4,168472207.0,"[ME12 4RP, ME12 4RL, ME12 4BS]","{'nudism': 'yes', 'natural': 'beach', 'name': ...","[51.3758713, 51.3767414, 51.3785798, 51.379209...","[0.9489995, 0.9476394, 0.9455351, 0.9450513, 0...",yes,beach,NaN,Leysdown East,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(POINT (0.9489995 51.3758713), POINT (0.947639...",NaN


In [125]:
ways_nodes.shape

(27, 41)

We dropped buffer and geometry, so all good

In [127]:
ways_nodes.nudism.value_counts()

yes           16
designated     4
no             2
permissive     2
customary      2
obligatory     1
Name: nudism, dtype: int64

In [130]:
ways_nodes[['name','note','nudism','postcode','lat','lon']]

,name,note,nudism,postcode,lat,lon
0,Brighton Beach,NaN,no,"[BN1 1GF, BN2 1TN, BN2 1TE, BN2 1TY, BN1 1EL, ...","[50.8170231, 50.8161311, 50.8187325, 50.819078...","[-0.1237390, -0.1201978, -0.1320089, -0.133979..."
1,Petit Tor Beach,NaN,permissive,[nan],"[50.4886730, 50.4879567, 50.4886730, 50.487741...","[-3.5144732, -3.5142425, -3.5144732, -3.514711..."
2,NaN,NaN,no,"[BN2 5UE, BN2 5TT, BN2 5TU, BN2 5TY, BN2 5UD, ...","[50.8134502, 50.8128533, 50.8133697, 50.813928...","[-0.1094882, -0.1075543, -0.1085115, -0.109789..."
3,Eastney Beach,NaN,yes,[PO4 9LJ],"[50.7868400, 50.7870341, 50.7870883, 50.786557...","[-1.0309351, -1.0308962, -1.0304885, -1.032078..."
4,Leysdown East,NaN,yes,"[ME12 4RP, ME12 4RL, ME12 4BS]","[51.3758713, 51.3767414, 51.3785798, 51.379209...","[0.9489995, 0.9476394, 0.9455351, 0.9450513, 0..."
5,St Osyth Naturist Beach,NaN,yes,[nan],"[51.7701689, 51.7702414, 51.7714794, 51.771184...","[1.0624927, 1.0674163, 1.0736186, 1.0729685, 1..."
6,Brighton Naturist Beach,NaN,yes,"[BN2 1FN, BN2 1FD, BN2 1FP, BN2 1FY]","[50.8143418, 50.8143418, 50.8149338, 50.815281...","[-0.1132645, -0.1132645, -0.1158513, -0.115545..."
7,Abbott's Farm (Naturist),NaN,yes,[nan],"[52.6759496, 52.6759691, 52.6759691, 52.677537...","[1.0071964, 1.0081888, 1.0081888, 1.0073828, 1..."
8,Heritage Club,NaN,yes,"[RG45 6BU, RG11 6BU, RG11 6BS, RG45 6BS]","[51.3696029, 51.3683716, 51.3685059, 51.367909...","[-0.8297808, -0.8317231, -0.8318042, -0.831352..."
9,Studland Naturist Beach,NaN,yes,[nan],"[50.6629957, 50.6606995, 50.6646572, 50.661110...","[-1.9487428, -1.9502858, -1.9475646, -1.950018..."


### Save data

In [131]:
ways_nodes.to_csv(FP_LOCAL,encoding='utf-8',index=False)